In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = False

In [15]:
import itertools

In [3]:
df = pd.read_csv('./Datos/Base_Rate_Share.csv', sep=';')

In [42]:
df.rename(columns={
    'RANGO_REC':'rangoRec', 
    'RANGO_ANT':'rangoAnt', 
    'SEGMENTACION':'segmentacion'  
}, inplace=True)

In [8]:
variables_agrupacion = ['SEGMENTACION']

In [9]:
def rate_share_grupos(df, variables_agrupacion):

    df_agrupado = df.groupby(variables_agrupacion).agg({
        'OPENING': ['sum'],
        'ENDING': ['sum'],
        'CHURN': ['sum']
    }).reset_index()

    variables_agrupacion.extend(['OPENING', 'ENDING', 'CHURN'])
    df_agrupado.columns = variables_agrupacion
    df_agrupado['BASE_MEDIA'] = (df_agrupado['OPENING'] + df_agrupado['ENDING'])/2
    df_agrupado['RATE'] = round((df_agrupado['CHURN'] / df_agrupado['BASE_MEDIA']) * 100, 2)
    df_agrupado['SHARE'] = df_agrupado['CHURN'].apply(lambda x: round(x/df_agrupado['CHURN'].sum()*100, 2))
    return df_agrupado

In [10]:
rate_share_grupos(df, variables_agrupacion)

,SEGMENTACION,OPENING,ENDING,CHURN,BASE_MEDIA,RATE,SHARE
0,1. Gross,0,246430,85694,123215.0,69.55,33.23
1,2. Recurrentes,333988,248145,85843,291066.5,29.49,33.29
2,3. Esporadicos,333887,247536,86351,290711.5,29.70,33.48


In [12]:
df.sample()

,SUBS_ID,RANGO_REC,RANGO_ANT,SEGMENTACION,OPENING,ENDING,CHURN
970217,15485669,4. Rango 10000,5. Rango >3 meses,3. Esporadicos,1,1,0


In [43]:
elementos = ['rangoRec', 'rangoAnt', 'segmentacion']

In [54]:
combinaciones = []

for i in range(3):
    for j in range(len(elementos)):
        resultados = [list(j) for j in list(itertools.combinations(elementos, j+1))]
        for k in resultados:
            formateo = ' '.join("_").join(k)
            combinaciones.append(formateo)
            
            print(k)

            df_segmentado = rate_share_grupos(df, k)

            df_segmentado.to_csv(f'./Datos/Sintetizados/{formateo}.csv', index=False)


['rangoRec']
['rangoAnt']
['segmentacion']
['rangoRec', 'rangoAnt']
['rangoRec', 'segmentacion']
['rangoAnt', 'segmentacion']
['rangoRec', 'rangoAnt', 'segmentacion']
['rangoRec']
['rangoAnt']
['segmentacion']
['rangoRec', 'rangoAnt']
['rangoRec', 'segmentacion']
['rangoAnt', 'segmentacion']
['rangoRec', 'rangoAnt', 'segmentacion']
['rangoRec']
['rangoAnt']
['segmentacion']
['rangoRec', 'rangoAnt']
['rangoRec', 'segmentacion']
['rangoAnt', 'segmentacion']
['rangoRec', 'rangoAnt', 'segmentacion']
